## A Numerical Example (Least Squares)

- We create a data matrix A with one column that is virtually dependent on the other two: by a small random vector (so as to make A's condition number reasonably large).
- we create a target column (the dependent vector in linear least squares) that differs from a linear combination of two data columns by a small random vector: the latter determines how far the target is from the range of A and hence, how large the residual is from the least squares solution

In [1]:
import numpy as np

c1 = np.array([1,2,4,8])
c2 = np.array([3,6,9,12])
c3 = c1 - 4*c2 + 0.0000001*(np.random.rand(4) - 0.5*np.ones(4).T)

In [2]:
A = np.matrix([c1,c2,c3]).T

In [3]:
A  # Matrix with data columns; column 3 is almost dependent on columns 1 and 2.

matrix([[  1.        ,   3.        , -10.99999998],
        [  2.        ,   6.        , -22.00000001],
        [  4.        ,   9.        , -32.00000005],
        [  8.        ,  12.        , -40.00000004]])

In [4]:
b = 2*c1 - 7*c2 + 0.0001*(np.random.rand(4) - 0.5*np.ones(4))    # Target Column

In [5]:
b = np.matrix(b).T

In [6]:
b.shape

(4, 1)

In [7]:
u, sigma, vt = np.linalg.svd(A)  # the singular value decomposition

In [8]:
u

matrix([[-0.19098134,  0.27860381,  0.84834319, -0.40770072],
        [-0.38196268,  0.55720763,  0.08310079,  0.73261069],
        [-0.55954094,  0.38729987, -0.50727238, -0.52876033],
        [-0.71031301, -0.67963106,  0.12681809,  0.13219008]])

The condition number of $A$ is $\sigma_1 / \sigma_3 \approx 10^{9}$. 

Hence, the condition number of $A^T A$, the ratio of its largest to smallest eigenvalue, is $\approx 10^{18}$.

In [9]:
sigma   # note disparity in the relative magitudes of the largest vs. smallest singular values

array([  5.98101360e+01,   2.59762139e+00,   8.64308580e-09])

In [10]:
u[:,:3]*(np.diag(sigma))*vt   # Full reconstruction

matrix([[  1.        ,   3.        , -10.99999998],
        [  2.        ,   6.        , -22.00000001],
        [  4.        ,   9.        , -32.00000005],
        [  8.        ,  12.        , -40.00000004]])

In [11]:
sigmap = np.linalg.pinv(np.diag(sigma))  # Moore-Penrose inverse

In [12]:
sigmap = np.matrix(sigmap)

In [13]:
sigmap

matrix([[  1.67195741e-02,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   3.84967572e-01,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   1.15699418e+08]])

In [14]:
vt.T

matrix([[-0.14839583, -0.96042861, -0.23570226],
        [-0.27460805, -0.18894849,  0.94280904],
        [ 0.95003637, -0.20463466,  0.23570226]])

Solve the linear least squares regression problem using the SVD of $A$, namely $A = U \Sigma V^T$:

$ x = V {\Sigma}^{+} U^T b$

where ${\Sigma}^{+}$ is the Moore-Penrose inverse of the diagonal matrix $\Sigma$.

In [15]:
r1 = b - A*(vt.T)*sigmap*(u[:,:3].T)*b

In [16]:
r1

matrix([[ -8.43690507e-06],
        [  1.51605501e-05],
        [ -1.09421045e-05],
        [  2.73551808e-06]])

In [17]:
np.linalg.norm(r1)

2.0693872391528605e-05

So the SVD approach yields a reasonable answer: we should expect a very close fit to the target $b$ and hence a very small residual error.

In [18]:
np.linalg.eigvalsh(A.T*A)

array([  1.33795267e-13,   6.74763688e+00,   3.57725237e+03])

Now solve the linear least squares regression problem using the formula derived from the normal equations:

$ x = (A^T A)^{-1} A^T b $

In [19]:
r2 = A*np.linalg.inv(A.T*A)*(A.T)*b

In [20]:
r2 - b

matrix([[  5.43090088],
        [ 10.86178517],
        [ 16.76734979],
        [-62.70597124]])

In [21]:
np.linalg.norm(r2 - b)

66.035262618442502

Notice the wild answer! This is due to the numerical instability of computing the inverse of $A^T A$. 